## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BSBSSSSS'
meas='TOBS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1473
+------------------+-----------------+------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+--------------------+-----------+-----------+------+--------------------+------+
|           coeff_1|          coeff_2|           coeff_3|elevation|   label|latitude|longitude|measurement|             res_1|             res_2|             res_3|            res_mean|    station|  total_var|undefs|              vector|  year|
+------------------+-----------------+------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+--------------------+-----------+-----------+------+--------------------+------+
| 717.5459612227579|96.80398671865137|-61.50489335794748|     56.4|BSBSSSSS| 30.7836| -83.5692|       TOBS|0.5299298728719344|0.5218511737391858|0.5159160510912769|0.061538323241307834|USC00

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [13]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_1'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_1 FROM weather


,station,year,coeff_1
0,USC00097276,1943.0,717.545961
1,USC00093578,2008.0,-845.673903
2,USC00013761,1961.0,-696.696052
3,USC00096087,1936.0,793.231405
4,USC00096087,1986.0,772.345715


In [14]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_1')
year_station_table.head(10)

station,USC00013761,USC00014895,USC00080025,USC00080804,USC00081020,USC00081022,USC00081356,USC00081544,USC00081601,USC00081775,...,USC00095585,USC00096038,USC00096087,USC00096356,USC00097165,USC00097276,USC00098666,USC00098703,USC00099519,USW00012832
year,,,,,,,,,,,,,,,,,,,,,
1901.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-400.011468,-863.593,NaN,NaN,-830.867682,NaN,NaN,NaN,NaN,NaN
1902.0,NaN,NaN,NaN,NaN,NaN,NaN,480.387232,NaN,NaN,NaN,...,-135.767877,NaN,NaN,NaN,-553.406308,NaN,NaN,NaN,NaN,NaN
1903.0,NaN,NaN,NaN,NaN,NaN,NaN,403.498879,NaN,NaN,NaN,...,-193.440575,NaN,NaN,NaN,-949.330043,NaN,NaN,NaN,NaN,NaN
1904.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-89.144186,NaN,NaN,NaN,-621.176570,NaN,NaN,NaN,NaN,NaN
1905.0,NaN,560.936555,NaN,NaN,NaN,NaN,382.419945,NaN,NaN,NaN,...,56.674724,NaN,NaN,NaN,-549.251505,NaN,NaN,NaN,NaN,NaN
1906.0,NaN,169.316892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-415.808302,NaN,NaN,NaN,NaN,NaN
1907.0,NaN,611.074551,NaN,NaN,NaN,NaN,43.967870,NaN,NaN,NaN,...,344.016390,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1908.0,NaN,834.694284,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,127.847719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909.0,NaN,960.603319,NaN,-429.867727,NaN,NaN,NaN,NaN,NaN,NaN,...,248.473740,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [15]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  674.043460101
RMS removing mean-by-station=  434.276787772
RMS removing mean-by-year   =  586.535794514


In [12]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 674.043460101
0 after removing mean by year    = 586.535794514
0 after removing mean by stations= 378.294965651
1 after removing mean by year    = 371.4810307
1 after removing mean by stations= 370.536845163
2 after removing mean by year    = 370.252973822
2 after removing mean by stations= 370.146138138
3 after removing mean by year    = 370.103182888
3 after removing mean by stations= 370.085451933
4 after removing mean by year    = 370.078019452
4 after removing mean by stations= 370.074864421
